# Analysis and Visualization of Complex Agro-Environmental Data
---
## Ordination Analysis

### 1. Principal Component Analysis (PCA)

PCA is one of the most popular methods in Exploratory Data Analysis, as a means to explore, through visualizations, the relationships among a set of objects (sampling units, indivíduals, experimental plots, ...) in relation to a set of variables (different measurements undertaken at each object). It is also often use to create few orthogonal latent variables that retain most of the original data variance. These latent variables are specially usefull in regression and machine learning, to avoid problems derived from multicolinearity among explanatory variables.

In `python` PCA is implemented in the `sklearn.cluster` module, in the `sklearn.decomposition.PCA` function (check [here](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html))

### 1.1 Data preparation and exploration

To run the analysis you first need to import necessary modules and functions:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

We will analyse the dataset `winequality_red.csv` available [here](https://www.kaggle.com/datasets/sgus1318/winedata?resource=download) with 12 variables (columns) related with wine quality characteristics. 

In [ ]:
df_wine = pd.read_csv('winequality_red.csv')
df_wine.head()

df_wine()['quality'],

Remove variable `quality`

In [ ]:
df = df_wine.iloc[:, 0:11]

Let's check multicolinearity in the dataset

In [ ]:
def corrdot(*args, **kwargs):
    corr_r = args[0].corr(args[1], 'pearson')
    corr_text = f"{corr_r:2.2f}".replace("0.", ".")
    ax = plt.gca()
    ax.set_axis_off()
    marker_size = abs(corr_r) * 10000
    ax.scatter([.5], [.5], marker_size, [corr_r], alpha=0.6, cmap="coolwarm",
               vmin=-1, vmax=1, transform=ax.transAxes)
    font_size = abs(corr_r) * 40 + 5
    ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction",
                ha='center', va='center', fontsize=font_size)

sns.set(style='white', font_scale=1.6)

g = sns.PairGrid(df, aspect=1.4, diag_sharey=False)
g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color': 'black'})
g.map_diag(sns.histplot, kde_kws={'color': 'black'})
g.map_upper(corrdot);

With a few exceptions, variables are not so correlated with each other. This means that probably it will be difficult that the first Principal Components will retain most of the variance in the data.

### 1.2 Data standardization

Now we need to scale the variables before conducting the analysis to avoid misleading PCA results due to units differences. The variables need to be standardize, so that the mean of each variable = 0 and the variance = 1. We will use the `StandardScaler` class of `scikit-learn.

In [ ]:

wine_scaled = StandardScaler().fit_transform(df)
# As a result, we obtained a two-dimensional NumPy array. We can convert it to a pandas DataFrame for a better display.
df_scaled = pd.DataFrame(data=wine_scaled, 
                                columns=df.columns)
df_scaled.head()

We are now ready to run PCA.

### 1.3 Select the number of Principal Components (PC)

To select the number of PC, usually a Scree plot is produced to visualize the percentage of explained variance or the eigenvalues per component. Either the `elbow rule` - retaining all components before the point where the curve flattens out - or the `Kaiser's rule` - retaining components whose eigenvalues are greater than 1.

First we run PCA for 11 components (number of original variables).

In [ ]:
pca = PCA(n_components=11)
pca.fit_transform(df_scaled)

We have run the PCA and now we can extract the proportion of explained variance and the eigenvalues as follows:

In [ ]:
eigenvalues = pca.explained_variance_ # eigenvalues
prop_var = pca.explained_variance_ratio_ # proportion of explained variance

Plot the scree plot with proportion of variance

In [ ]:
PC_numbers = np.arange(pca.n_components_) + 1
 
plt.plot(PC_numbers, 
         prop_var,
         'ro-')
plt.title('Scree Plot', fontsize=20)
plt.ylabel('Proportion of Variance', fontsize=16)
plt.show()

The `Elbow rule` is not easy to apply to our data.
We can try to apply the `Kaiser's rule` and for that we need to plot the eigenvalues instead.

In [ ]:
PC_numbers = np.arange(pca.n_components_) + 1
 
plt.plot(PC_numbers, 
         eigenvalues,
         'ro-')
plt.title('Scree Plot', fontsize=20)
plt.ylabel('Eigenvalues', fontsize=16)
plt.axhline(y=1, color='r', 
            linestyle='--')
plt.show()

According to this rule, 4 components should be retained. However, just for the purpose of ilustrating how to produce a PCA biplot vlisualization, we will retain only two components.

### 1.4 Visualize and interpret the PCA biplot

In [ ]:
pca = PCA(n_components=2)
PC = pca.fit_transform(df_scaled)

In [ ]:
pca_wine = pd.DataFrame(data = PC, 
                            columns = ['PC1', 'PC2'])
pca_wine.head(6)

In [ ]:
# "score" - Scores of each object (for each PC) - PC
# "coef" - PCA variable loadings (for each PC) - pca.components_
# labels - name of variables - list(df.columns)


def biplot(score,coef,labels=None): 
 
    xs = score[:,0] # PC1 object scores
    ys = score[:,1] # PC2 object scores 
    n = coef.shape[0] # number of dimensions (2)
    scalex = 1.0/(xs.max() - xs.min()) # to rescale scores
    scaley = 1.0/(ys.max() - ys.min()) # to rescale scores
    plt.scatter(xs * scalex,ys * scaley,
                s=6, 
                color='blue') # scatter plot using rescaled object scores
 
    for i in range(n):
        plt.arrow(0, 0, coef[i,0], 
                  coef[i,1],color = 'red',
                  head_width=0.01,
                  alpha = 0.5) # plot arrows for each variable
        plt.text(coef[i,0]* 1.15, 
                 coef[i,1] * 1.15, 
                 labels[i], 
                 color = 'red', 
                 ha = 'center', 
                 va = 'center') # variable labels for each arrow
 
    plt.xlabel("PC{}".format(1))
    plt.ylabel("PC{}".format(2))    
 
 
    plt.figure()

In [ ]:
plt.figure(figsize=(10, 8))
plt.title('Biplot of PCA')
 
biplot(PC, 
       np.transpose(pca.components_), 
       list(df.columns))
plt.show()

In [ ]:
# same biplot but with colors expressing wine quality

PC1 = pca_wine['PC1']/(pca_wine['PC1'].max() - pca_wine['PC1'].min())
PC2 = pca_wine['PC2']/(pca_wine['PC2'].max() - pca_wine['PC2'].min())

plt.figure(figsize=(10, 8))
plt.title('Biplot of PCA')
sns.scatterplot(x=PC1,
              y=PC2,
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )

n = np.transpose(pca.components_).shape[0] # number of dimensions (2)
for i in range(n):
        plt.arrow(0, 0, np.transpose(pca.components_)[i,0], 
                  np.transpose(pca.components_)[i,1], 
                  color = (0.1, 0.1, 0.1, 0.8),
                  head_width=0.02) # plot arrows for each variable
        plt.text(np.transpose(pca.components_)[i,0]* 1.15, 
                 np.transpose(pca.components_)[i,1] * 1.15, 
                 list(df.columns)[i], 
                 color = (0.1, 0.1, 0.1, 0.8), 
                 ha = 'center', 
                 va = 'center') # variable labels for each arrow
plt.legend(title='Wine quality')
plt.xlabel("PC{}".format(1))
plt.ylabel("PC{}".format(2))

plt.show()


### 1.5 Running PCA from scratch

The code that follows computes the same PCA but without depending on any specific PCA function.

NOTE: The first step - variable standardization - was already performed

Step 2: compute the covariance matrix 

In [ ]:
def covariance(x): 
    return (x.T @ x)/(x.shape[0]-1)

cov_mat = covariance(df_scaled) # np.cov(X_std.T)

Step 3: Compute the eigenvectors and eigenvalues of the covariance matrix

In [ ]:
from numpy.linalg import eig

# Eigendecomposition of covariance matrix
eig_vals, eig_vecs = eig(cov_mat)

# Adjusting the eigenvectors (loadings) that are largest in absolute value to be positive
max_abs_idx = np.argmax(np.abs(eig_vecs), axis=0)
signs = np.sign(eig_vecs[max_abs_idx, range(eig_vecs.shape[0])])
eig_vecs = eig_vecs*signs[np.newaxis,:]
eig_vecs = eig_vecs.T

print('Eigenvalues \n', eig_vals)
print('Eigenvectors \n', eig_vecs)

Step 4: Rearrange the eigenvectors and eigenvalues

In [ ]:
# Create a list of eigenvalue and eigenvector tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[i,:]) for i in range(len(eig_vals))]

# Sort the tuples from the highest to the lowest eigenvalues
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# For further usage
eig_vals_sorted = np.array([x[0] for x in eig_pairs])
eig_vecs_sorted = np.array([x[1] for x in eig_pairs])


Step 5: Select the number of PCs

In [ ]:
# Select top k eigenvectors
k = 2
W = eig_vecs_sorted[:k, :] # Projection matrix

eig_vals_total = sum(eig_vals)
explained_variance = [(i / eig_vals_total)*100 for i in eig_vals_sorted]
explained_variance = np.round(explained_variance, 2)
cum_explained_variance = np.cumsum(explained_variance)

print('Explained variance: {}'.format(explained_variance))
print('Cumulative explained variance: {}'.format(cum_explained_variance))

plt.plot(np.arange(1, 11 + 1), cum_explained_variance, '-o')
plt.xticks(np.arange(1,11+1))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance');
plt.show()

Step 6: Project the data with a biplot

In [ ]:
# 
X_proj = df_scaled.dot(W.T)


In [ ]:
# Create biplot
PC1 = X_proj.iloc[:, 0]/(X_proj.iloc[:, 0].max() - X_proj.iloc[:, 0].min())
PC2 =X_proj.iloc[:, 1]/(X_proj.iloc[:, 1].max() - X_proj.iloc[:, 1].min())

plt.figure(figsize=(10, 8))
plt.title('2 components, captures {:.1f} of total variation'.format(cum_explained_variance[1]))

sns.scatterplot(x=PC1,
              y=PC2,
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )

n = np.transpose(pca.components_).shape[0] # number of dimensions (2)
for i in range(n):
        plt.arrow(0, 0, np.transpose(pca.components_)[i,0], 
                  np.transpose(pca.components_)[i,1], 
                  color = (0.1, 0.1, 0.1, 0.8),
                  head_width=0.02) # plot arrows for each variable
        plt.text(np.transpose(pca.components_)[i,0]* 1.15, 
                 np.transpose(pca.components_)[i,1] * 1.15, 
                 list(df.columns)[i], 
                 color = (0.1, 0.1, 0.1, 0.8), 
                 ha = 'center', 
                 va = 'center') # variable labels for each arrow
plt.legend(title='Wine quality')
plt.xlabel("PC{}".format(1))
plt.ylabel("PC{}".format(2))

plt.xlabel('PC1'); plt.xticks([])
plt.ylabel('PC2'); plt.yticks([])
plt.show()

### 1.6 Running PCA based on the correlation matrix

Running PCA based on the correlation matrix gives the same result as running a PCA based on the variance-covariance matrix with standardized data.

In [ ]:
# In the PCA function, by setting the whiten=True parameter when initializing the PCA object, 
# the covariance matrix is normalized by dividing each element by the square root of the corresponding product of variances. 
# This normalization results in a correlation matrix.

pca = PCA(n_components=11, whiten=True)
PC = pca.fit_transform(df_scaled)
pca_wine = pd.DataFrame(data = PC[:,:2], columns = ['PC1', 'PC2'])

PC1 = pca_wine['PC1']/(pca_wine['PC1'].max() - pca_wine['PC1'].min())
PC2 = pca_wine['PC2']/(pca_wine['PC2'].max() - pca_wine['PC2'].min())

plt.figure(figsize=(10, 8))
plt.title('Biplot of PCA')
sns.scatterplot(x=PC1,
              y=PC2,
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )
              
n = np.transpose(pca.components_).shape[0] # number of dimensions (2)
for i in range(n):
        plt.arrow(0, 0, np.transpose(pca.components_)[i,0], 
                  np.transpose(pca.components_)[i,1], 
                  color = (0.1, 0.1, 0.1, 0.8),
                  head_width=0.02) # plot arrows for each variable
        plt.text(np.transpose(pca.components_)[i,0]* 1.15, 
                 np.transpose(pca.components_)[i,1] * 1.15, 
                 list(df.columns)[i], 
                 color = (0.1, 0.1, 0.1, 0.8), 
                 ha = 'center', 
                 va = 'center') # variable labels for each arrow
plt.legend(title='Wine quality')
plt.xlabel("PC{}".format(1))
plt.ylabel("PC{}".format(2))

plt.show()


### 1.7 Running Factor Analysis with varimax rotation

Varimax rotation allows to reorient the factors so that low loadings become lower and high loadings larger. This facilitates the interpretation of the PC's in terms of their correlation with the variables. In Sklearn, the Varimax is not implemented in `sklearn.decomposition.PCA`, only in `sklearn.decomposition.FactorAnalysis`. Although similar, Factor Analysis is a slightly different approach from PCA. While PCA is a data reduction technique that aims to capture the most variance in the data, FA is a technique for identifying latent factors that underlie the observed correlations among the variables.


In [ ]:
from sklearn.decomposition import FactorAnalysis

# Fit factor analysis model
favarimax = FactorAnalysis(n_components=2, rotation="varimax")
FA = favarimax.fit_transform(df_scaled)

fa_wine = pd.DataFrame(data = FA[:,:2], columns = ['F1', 'F2']) # select 2 first array columns from FA

F1 = fa_wine['F1']/(fa_wine['F1'].max() - fa_wine['F1'].min())
F2 = fa_wine['F2']/(fa_wine['F2'].max() - fa_wine['F2'].min())

plt.figure(figsize=(10, 8))
plt.title('Biplot of FA')
sns.scatterplot(x=F1,
              y=F2,
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )
              
n = np.transpose(favarimax.components_).shape[0] # number of dimensions (2)
for i in range(n):
        plt.arrow(0, 0, np.transpose(favarimax.components_)[i,0], 
                  np.transpose(favarimax.components_)[i,1], 
                  color = (0.1, 0.1, 0.1, 0.8),
                  head_width=0.02) # plot arrows for each variable
        plt.text(np.transpose(favarimax.components_)[i,0]* 1.15, 
                 np.transpose(favarimax.components_)[i,1] * 1.15, 
                 list(df.columns)[i], 
                 color = (0.1, 0.1, 0.1, 0.8), 
                 ha = 'center', 
                 va = 'center') # variable labels for each arrow
plt.legend(title='Wine quality')
plt.xlabel("PC{}".format(1))
plt.ylabel("PC{}".format(2))

plt.show()



### 2. Multidimensional Scaling (MDS) and Non-Metric Multidimensional Scaling

The aim of this ordination method is to project objects from a higher-dimensional space to a lower-dimensional space while preserving the relative distances between those points as much as possible. PCoA (also known as MDS) can be viewed as a generalization of PCA that allows a much wider range of dissimilarity measures to be used (dissimilarities among objects in PCA are euclidean distances).

MDS and NMDS are implemented in the `mds()` function of `sklearn.manifold` module.

#### 2.1 Import modules and functions

Import necessary modules and functions:

In [ ]:
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

#### 2.2 Run MDS

We will run MDS using the same wine dataset used in PCA. We will use the standardized data (df_scaled).

In [ ]:
# by default the 'mds' argument is set to 'True', which means it will run a MDS
# Euclidean distances are also the default 
# Only two axis are extracted by default
mds = MDS(random_state=0, normalized_stress = False) 
mds_transf = mds.fit_transform(df_scaled)

Plot the result

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=mds_transf[:,0],
              y=mds_transf[:,1],
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )
plt.show()

We may use other distances to compute MDS. For that we need to run the function using a dissimilarity matrix as the input.
Let's try running MDS with Manhattan dissimilarities.

In [ ]:
# compute MDS
dist_manhattan = manhattan_distances(df_scaled)
mds = MDS(dissimilarity='precomputed', random_state=0, normalized_stress = False)
mds_transf2 = mds.fit_transform(dist_manhattan)

In [ ]:
# plot the result in 2D

plt.figure(figsize=(10, 6))
sns.scatterplot(x=mds_transf2[:,0],
              y=mds_transf2[:,1],
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )
plt.show()

#### 2.3 Run NMDS

Non-Metric Multidimensional Scaling solved many issues of MDS, namely the computation of the measure used to evaluate how much the distances in the MDS are related with the original dissimilarities. This methods is based on ranks of distances among objects, so the aim is to retain the relative position of objects from each other, not their distances. It is based on a iterative trial & error algorithm. 

In [ ]:
# Compute NMDS
nmds = MDS(n_components=5, random_state=0, metric = False, normalized_stress="auto") # 5 components extracted so that stress is > 0.2
nmds_transf = nmds.fit_transform(df_scaled)

The match between object distance in NMDS and the original dissimilarities is given by the stress, which is proportional to the residuals of that relationship. The stress of an MDS with a perfect match between MDS distances and original dissimilarities will be closer to zero. 

As a rule of thumb, an NMDS ordination with a stress value around or above 0.2 is deemed suspect and a stress value approaching 0.3 indicates that the ordination is arbitrary. Stress values equal to or below 0.1 are considered fair, while values equal to or below 0.05 indicate good fit.

In [ ]:
# Compute the stress
stress = nmds.stress_
print(stress)

In [ ]:
# plot the result

plt.figure(figsize=(10, 6))
sns.scatterplot(x=nmds_transf[:,0],
              y=nmds_transf[:,1],
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )
plt.show()

It is also possible to use the method with dissimilarities other than Euclidean distances.

In [ ]:
# NMDS based on Manhattan distances
nmds2 = MDS(metric = False, random_state=0, normalized_stress = 'auto', dissimilarity='precomputed')
nmds_transf2 = nmds2.fit_transform(dist_manhattan)

plt.figure(figsize=(10, 6))
sns.scatterplot(x=nmds_transf2[:,0],
              y=nmds_transf2[:,1],
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )
plt.show()

In [ ]:
stress = nmds2.stress_
print(stress)

### 3. Discriminant Analysis

Discriminant Analysis (DA) is used when we have observations from a pre-determined groups with two or more variables recorded for each observation. It may be used as an ordination method, by displaying objects such that the discrimination among groups is maximized using the fewest dimensions as possible, or a classification method, by generating linear combination of variables that maximizes the probability of correctly assigning observations to groups. The aim is also to assess which variables most contribute to the discrimination among pre-determined groups.

The two most common Discriminant Analysis methods are the Linear Discriminant Analysis (LDA) and the Quadratic Discriminant Analysis (QDA). LDA can only learn linear boundaries, while QDA can learn quadratic boundaries and is therefore more flexible. They are implemented respectively in `sklearn.discriminant_analysis.LinearDiscriminantAnalysis()` and `sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis()` functions of the the `sklearn` module. Here, we will give an example of Linear Discriminant Analysis

#### 3.1 Linear Discriminant Analysis

Import necessary modules and functions:

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis 
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier


Define predictor and response variables

In [ ]:

X = df_scaled
y = df_wine['quality'] # quality has 6 classes (3,4,5,6,7,8)

#Fit the LDA model (we set to two components just for ilustration - default would be N_classes-1=5 in this case)
model = LinearDiscriminantAnalysis(n_components=2)
DLA = model.fit_transform(X, y)

Some atributes

In [ ]:
# Weight vectors of variables for each class
model.coef_

In [ ]:
# Proportion of explained variance of the selected components
model.explained_variance_ratio_

In [ ]:
# Contribution of each variable (row) to each Component (column)
model.scalings_ 

Extract the first two discriminant axis to a DataFrame

In [ ]:
DLA_scores = pd.DataFrame(data = DLA, 
                            columns = ['LD1', 'LD2'])
DLA_scores.head(6)

Define method to evaluate model

In [ ]:
#defines the kfold crossvalidation settings for the next function 'cross_val_score'
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1) 

#evaluate model (classification accuracy - from 0 to 1)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print(np.mean(scores))

Plot the fist discriminant plane

In [ ]:
plt.figure(figsize=(10, 8))

sns.scatterplot(x=DLA_scores['LD1'],
              y=DLA_scores['LD2'],
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )
plt.show()

Biplot

In [ ]:
plt.figure(figsize=(10, 8))

sns.scatterplot(x=DLA_scores['LD1'],
              y=DLA_scores['LD2'],
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )

n = model.n_features_in_
for i in range(11):
        plt.arrow(0, 0, model.scalings_[i,0]*4, # Scalings were multiplied by a factor of 4 to just to facilitate the visualization
                  model.scalings_[i,1]*4, # Scalings were multiplied by a factor of 4 to just to facilitate the visualization
                  color = (0.1, 0.1, 0.1, 0.8),
                  head_width=0.02) # plot arrows for each variable
        plt.text(model.scalings_[i,0]* 4.3, # plot the names of the variables
                 model.scalings_[i,1] * 4.3,
                 list(df.columns)[i], 
                 color = (0.1, 0.1, 0.1, 0.8), 
                 ha = 'center', 
                 va = 'center') # variable labels for each arrow

plt.xlabel('LD1')
plt.ylabel('LD2')
plt.show()


Same plot but showing predictive classification areas (voronoi plane):

In [ ]:
fig = plt.figure(figsize=(8,8))
ax0 = fig.add_subplot(111)
ax0.set_xlim(-4,6)
ax0.set_ylim(-7,4.2)

colors = list(sns.color_palette("Spectral_r", 6).as_hex())

# Plot the voronoi spaces

means = []
for target in zip(np.unique(y)):
    means.append(np.mean(DLA_scores[y==target],axis=0))
    
mesh_x, mesh_y = np.meshgrid(np.linspace(-4,6,200),np.linspace(-7,4.2,200)) # creates a grid of points to which we apply class predictions using the Linear discrimant model
mesh = [] # creates an empty mesh

for i in range(len(mesh_x)):
    for j in range(len(mesh_x[0])):
        date = [mesh_x[i][j],mesh_y[i][j]]
        mesh.append((mesh_x[i][j],mesh_y[i][j]))

NN = KNeighborsClassifier(n_neighbors=1) # set the k-nearest neighbor classifier to 1 nearest neighbor
NN.fit(means,colors) # fit the 1 nearest neighbor
predictions = NN.predict(np.array(mesh)) # Predict the class labels for the provided data (grid of points) 
ax0.scatter(np.array(mesh)[:,0],np.array(mesh)[:,1],color=predictions,alpha=0.1, linewidths=0 ) # plots the grid with the predicted class (color)


sns.scatterplot(x=DLA_scores['LD1'],
              y=DLA_scores['LD2'],
              hue = df_wine['quality'].tolist(),
              palette=sns.color_palette("Spectral_r", 6),
              linewidth=0.5,
              edgecolor='black'
              )

plt.legend(loc='upper right')

n = model.n_features_in_
for i in range(11):
        plt.arrow(0, 0, model.scalings_[i,0]*4, # Scalings were multiplied by a factor of 4 to just to facilitate the visualization
                  model.scalings_[i,1]*4, # Scalings were multiplied by a factor of 4 to just to facilitate the visualization
                  color = (0.1, 0.1, 0.1, 0.8),
                  head_width=0.02) # plot arrows for each variable
        plt.text(model.scalings_[i,0]* 4.3, # plot the names of the variables
                 model.scalings_[i,1] * 4.3,
                 list(df.columns)[i], 
                 color = (0.1, 0.1, 0.1, 0.8), 
                 ha = 'center', 
                 va = 'center') # variable labels for each arrow


for target, c in zip(np.unique(y), colors):
    ax0.scatter(np.mean(DLA_scores[y==target],axis=0)[0],
                np.mean(DLA_scores[y==target],axis=0)[1],
                c=c,s=150, linewidths=2, edgecolor='black'
                )


plt.show()


Let's plot some boxplots of the variables that seems more correlated with the Discriminant axis.

In [ ]:
fig, axs = plt.subplots(2, 1, sharex=True, figsize=(6, 6))

sns.boxplot(x='quality', y='alcohol', data = df_wine, ax=axs[0])
sns.boxplot(x='quality', y='volatile acidity', data = df_wine, ax=axs[1])
plt.show()


### 1.5 Running DA from scratch

The code that follows computes the same LDA but without depending on any specific function.

Step 0: Load in the data and split the descriptive and the target feature

In [ ]:
X = df
y = df_wine['quality'] # quality has 6 classes (3,4,5,6,7,8)

Step 1: Standardize the data

In [ ]:
for col in X.columns:
    X[col] = StandardScaler().fit_transform(X[col].values.reshape(-1,1))

Step2: Compute the mean vector mu and the mean vector per class mu_k

In [ ]:
mu = np.mean(X,axis=0).values.reshape(11,1) # Mean vector mu --> Since the data has been standardized, the data means are zero 

mu_k = []
for i, wine in enumerate(np.unique(df_wine['quality'])):
    mu_k.append(np.mean(X.where(df_wine['quality']==wine),axis=0))
mu_k = np.array(mu_k).T

Step 3: Compute the Scatter within and Scatter between matrices

In [ ]:
data_SW = []
Nc = []
for i,orchid in enumerate(np.unique(df_wine['quality'])):
    a = np.array(X.where(df_wine['quality']==orchid).dropna().values-mu_k[:,i].reshape(1,11))
    data_SW.append(np.dot(a.T,a))
    Nc.append(np.sum(df_wine['quality']==orchid))
SW = np.sum(data_SW,axis=0)

SB = np.dot(Nc*np.array(mu_k-mu),np.array(mu_k-mu).T)

Step 4: Compute the Eigenvalues and Eigenvectors of SW^-1 SB

In [ ]:
eigval, eigvec = np.linalg.eig(np.dot(np.linalg.inv(SW),SB))

Step 5: Select the two largest eigenvalues 

In [ ]:
eigen_pairs = [[np.abs(eigval[i]),eigvec[:,i]] for i in range(len(eigval))]
eigen_pairs = sorted(eigen_pairs,key=lambda k: k[0],reverse=True)
w = np.hstack((eigen_pairs[0][1][:,np.newaxis].real,eigen_pairs[1][1][:,np.newaxis].real)) # Select two largest

Step 6: Transform the data with Y=X*w

In [ ]:
Y = X.dot(w)

Step 7: Plot 1st discriminant plane, showing predictive classification areas (voronoi plane).

In [ ]:
# Plot
fig = plt.figure(figsize=(8,8))
ax0 = fig.add_subplot(111)
ax0.set_xlim(-6,4)
ax0.set_ylim(-3,4)

colors = list(sns.color_palette("Spectral_r", 6).as_hex())

# Plot the voronoi spaces

means = []
for target in zip(np.unique(y)):
    means.append(np.mean(Y[y==target],axis=0))

mesh_x, mesh_y = np.meshgrid(np.linspace(-6,4,200),np.linspace(-3,4,200)) # creates a grid of points to which we apply class predictions using the Linear discrimant model
mesh = [] # creates an empty mesh

for i in range(len(mesh_x)):
    for j in range(len(mesh_x[0])):
        date = [mesh_x[i][j],mesh_y[i][j]]
        mesh.append((mesh_x[i][j],mesh_y[i][j]))

NN = KNeighborsClassifier(n_neighbors=1) # set the k-nearest neighbor classifier to 1 nearest neighbor
NN.fit(means,colors) # fit the 1 nearest neighbor
predictions = NN.predict(np.array(mesh)) # Predict the class labels for the provided data (grid of points) 
ax0.scatter(np.array(mesh)[:,0],np.array(mesh)[:,1],color=predictions,alpha=0.05, linewidths=0 ) # plots the grid with the predicted class (color)

 
# Plot the data
for l,c in zip(np.unique(y),colors):
    ax0.scatter(Y[0][y==l],
                Y[1][y==l],
               c=c, label=l, linewidths=0.5, edgecolor='black')
ax0.legend(loc='upper right')

# plot the means (centroids)
means = []

for target, c in zip(np.unique(y), colors):
    ax0.scatter(np.mean(Y[y==target],axis=0)[0],
                np.mean(Y[y==target],axis=0)[1],
                c=c,s=150, linewidths=2, edgecolor='black')

plt.show()

### References

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html

https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html

https://scikit-learn.org/stable/modules/lda_qda.html

https://statisticsglobe.com/biplot-pca-python

https://python-course.eu/machine-learning/linear-discriminant-analysis-in-python.php

https://www.mltut.com/linear-discriminant-analysis-python-complete-and-easy-guide/

https://stackabuse.com/guide-to-multidimensional-scaling-in-python-with-scikit-learn/